# **13.7 Debugging_Techniques**

Debugging is the art of finding and fixing the bugs in your code. Every programmer — from beginner to expert — spends a significant portion of their time debugging. The good news is that debugging is a learnable skill with a systematic toolkit. In this lesson you'll learn proven strategies for hunting down bugs in your Pokemon programs: from print-based inspection all the way to Python's built-in debugger.

---

## **The Debugging Mindset**

Before reaching for any tool, good debugging starts with a clear process. Rushing straight to changing code often wastes time — systematic reasoning finds bugs much faster.

In [ ]:
# The debugging process — follow these steps every time

process = """
1. REPRODUCE — make the bug happen reliably
   "What exact input causes this?"

2. ISOLATE — find the smallest piece of code that shows the bug
   "Which function? Which line?"

3. UNDERSTAND — figure out WHY it's wrong
   "What is the code actually doing vs what I expected?"

4. FIX — change the code
   "What is the correct logic?"

5. VERIFY — confirm the fix works and didn't break anything else
   "Does it pass all test cases?"
"""
print(process)

---

## **Technique 1: print() Debugging**

The simplest and most widely used technique — add `print()` statements to inspect the state of variables at key points. It's fast, requires no setup, and works everywhere. The key is being *strategic* about where you print and what you print.

In [ ]:
# Buggy function — result is wrong
def calculate_team_average_level(team):
    total = 0
    for pokemon in team:
        total = pokemon['level']   # BUG: should be +=, not =
    return total / len(team)

team = [
    {"name": "Pikachu",  "level": 25},
    {"name": "Charizard","level": 36},
    {"name": "Blastoise","level": 36},
]

result = calculate_team_average_level(team)
print(f"Average level: {result}")   # Prints 12.0 — wrong! Should be 32.3

# Add print debugging to find the bug
def calculate_team_average_level_debug(team):
    total = 0
    for i, pokemon in enumerate(team):
        total = pokemon['level']   # Print lets us see total shrinking!
        print(f"  After {pokemon['name']}: total={total}")  # ← DEBUG PRINT
    return total / len(team)

print("\nWith debug prints:")
result = calculate_team_average_level_debug(team)
print(f"Result: {result}")  # Now we can see total gets OVERWRITTEN each loop

---

## **Technique 2: Inspecting Variables with type() and dir()**

When a variable has an unexpected value, checking its type and available attributes can reveal the problem instantly — especially when you receive data from functions you didn't write or from external sources like files and APIs.

In [ ]:
# Suppose you receive Pokemon data and it's behaving oddly
def diagnose_variable(name, value):
    """Print diagnostic info about any variable — useful during debugging."""
    print(f"\n--- Diagnosing '{name}' ---")
    print(f"  Value  : {value!r}")
    print(f"  Type   : {type(value).__name__}")
    if hasattr(value, '__len__'):
        print(f"  Length : {len(value)}")
    if isinstance(value, dict):
        print(f"  Keys   : {list(value.keys())}")

# Simulate receiving ambiguous data from a file
mystery_level = "25"           # Is it a string or int?
mystery_team  = None           # Did the load succeed?
mystery_stats = {"hp": 35}    # Is 'attack' in there?

diagnose_variable("mystery_level", mystery_level)
diagnose_variable("mystery_team",  mystery_team)
diagnose_variable("mystery_stats", mystery_stats)

# Now we can see: mystery_level is a str, not int — explains arithmetic errors!
print(f"\n'mystery_level' + 5 = ?")  
try:
    print(mystery_level + 5)
except TypeError as e:
    print(f"TypeError: {e}  ← expected, now we know to convert with int()")

---

## **Technique 3: Reading Tracebacks Carefully**

Python's traceback is your first and most important debugging tool. Most beginners glance at it and feel overwhelmed — but it tells you exactly where the problem is. Read it from the bottom up.

In [ ]:
import traceback

# Capture and display a traceback programmatically
def outer_call():
    return inner_call()

def inner_call():
    pokemon = {"name": "Pikachu", "level": 25}
    return pokemon['hp']   # KeyError — 'hp' not in dict

try:
    outer_call()
except KeyError:
    # Print the full traceback as a string for inspection
    tb = traceback.format_exc()
    print("--- Full Traceback ---")
    print(tb)

    print("--- How to Read It ---")
    print("1. Read from the BOTTOM UP")
    print("2. Bottom line = error type and message → 'KeyError: hp'")
    print("3. Line above = exact file and line that failed")
    print("4. Keep reading up to trace the call chain")

---

## **Technique 4: Rubber Duck Debugging**

Explain your code out loud — to a rubber duck, a colleague, or written as a comment. The act of verbalising forces you to think through the logic step by step, and bugs often reveal themselves in the process.

In [ ]:
# Buggy code — try explaining what it does line by line
def find_strongest_pokemon(team):
    """Return the Pokemon with the highest level."""
    strongest = None
    for pokemon in team:
        if strongest is None or pokemon['level'] > strongest:  # BUG!
            strongest = pokemon
    return strongest

# Explaining it out loud:
# "I set strongest to None.
#  For each pokemon: if strongest is None OR pokemon level > strongest...
#  Wait — strongest is the pokemon DICT not the level!
#  I'm comparing a level (int) to a dict. That's the bug."

# Fixed version:
def find_strongest_pokemon_fixed(team):
    """Return the Pokemon with the highest level."""
    strongest = None
    for pokemon in team:
        if strongest is None or pokemon['level'] > strongest['level']:  # FIXED
            strongest = pokemon
    return strongest

team = [
    {"name": "Pikachu",  "level": 25},
    {"name": "Charizard","level": 36},
    {"name": "Rattata",  "level": 8},
]

print(find_strongest_pokemon_fixed(team))

---

## **Technique 5: Isolating with Minimal Test Cases**

When a bug only appears in complex code, simplify. Create the smallest possible input that reproduces the problem. This strips away noise and makes the bug obvious.

In [ ]:
# Complex function — hard to see the bug
def process_battle_data(team, opponent_team, weather, turn_count):
    results = []
    for pokemon in team:
        for opp in opponent_team:
            damage = pokemon['attack'] * opp['defense']  # BUG: should be /
            results.append((pokemon['name'], opp['name'], damage))
    return results

# ISOLATE: strip away everything except the suspected bug
# Minimal test case:
p1 = {"name": "Pikachu",  "attack": 55, "defense": 40}
p2 = {"name": "Onix",     "attack": 45, "defense": 160}

# Just the buggy line:
damage = p1['attack'] * p2['defense']  # 55 * 160 = 8800 — way too high!
print(f"Bug: damage = {damage}")

# Correct:
damage_fixed = p1['attack'] / p2['defense'] * 50  # More realistic formula
print(f"Fix: damage = {damage_fixed:.1f}")

---

## **Technique 6: The Python Debugger (pdb)**

`pdb` is Python's built-in interactive debugger. It lets you pause execution at any line, inspect variables, step through code one line at a time, and even change values on the fly. Powerful for complex bugs that print statements can't easily track.

In [ ]:
# In a normal Python script you'd use:
#   import pdb; pdb.set_trace()   # Old style — pauses here
#   breakpoint()                  # Python 3.7+ shorthand

# In Jupyter notebooks, use %debug magic after an exception

# Key pdb commands:
pdb_commands = """
pdb Commands (type inside the debugger):

  n  (next)     — run next line, don't enter functions
  s  (step)     — step INTO the next function call
  c  (continue) — run until next breakpoint
  q  (quit)     — exit debugger
  p expr        — print the value of expr  (e.g. p pokemon)
  pp expr       — pretty-print (e.g. pp team)
  l  (list)     — show surrounding code
  w  (where)    — show call stack
  u / d         — move up/down the call stack
  h  (help)     — show all commands
"""
print(pdb_commands)

# Demonstrate: manually simulate what pdb shows
def buggy_heal(pokemon, amount):
    print(f"[pdb] p pokemon → {pokemon}")
    print(f"[pdb] p amount  → {amount}")
    # breakpoint()   # Uncomment to actually use pdb
    pokemon['hp'] = min(pokemon['max_hp'], pokemon['hp'] + amount)
    return pokemon

p = {"name": "Pikachu", "hp": 20, "max_hp": 35}
result = buggy_heal(p, 20)
print(f"After heal: {result}")

---

## **Technique 7: Logging**

The `logging` module is a professional alternative to print debugging. It gives you severity levels, timestamps, and the ability to turn output on/off without removing the statements — perfect for a game you want to monitor in production.

In [ ]:
import logging

# Configure logging — set level to DEBUG to see everything
logging.basicConfig(
    level=logging.DEBUG,
    format='%(levelname)-8s %(message)s'
)

logger = logging.getLogger('pokemon_game')

def battle_turn(attacker: dict, defender: dict) -> int:
    """Execute one battle turn with full logging."""
    logger.debug(f"Turn start: {attacker['name']} vs {defender['name']}")
    
    try:
        damage = attacker['attack'] * 25 // 10
        logger.debug(f"Damage calculated: {damage}")
        
        defender['hp'] = max(0, defender['hp'] - damage)
        logger.info(f"{attacker['name']} dealt {damage} damage — "
                    f"{defender['name']} HP: {defender['hp']}")
        
        if defender['hp'] == 0:
            logger.warning(f"{defender['name']} fainted!")
        
        return damage
    
    except KeyError as e:
        logger.error(f"Missing battle data: {e}")
        return 0

pikachu = {"name": "Pikachu",  "attack": 55, "hp": 35}
rattata = {"name": "Rattata",  "attack": 56, "hp": 10}

battle_turn(pikachu, rattata)
battle_turn(pikachu, rattata)   # Will faint

---

## **Common Bug Patterns in Pokemon Code**

In [ ]:
# Bug Pattern 1: Off-by-one error
team = ["Pikachu", "Charizard", "Blastoise"]
# for i in range(1, len(team) + 1):  # Bug: range(1, 4) → i = 1,2,3
#     print(team[i])                  # team[3] → IndexError!
for i in range(len(team)):            # Fix: range(0, 3) → i = 0,1,2
    print(f"{i}: {team[i]}")

# Bug Pattern 2: Mutable default argument
def bad_add_to_team(pokemon, team=[]):  # Shared list across all calls!
    team.append(pokemon)
    return team

def good_add_to_team(pokemon, team=None):  # Fix: use None as default
    if team is None:
        team = []
    team.append(pokemon)
    return team

print(good_add_to_team("Pikachu"))
print(good_add_to_team("Charizard"))   # Separate lists — correct

# Bug Pattern 3: Modifying list while iterating
team = [{"name": "Pikachu", "hp": 0},
        {"name": "Charizard", "hp": 78},
        {"name": "Blastoise", "hp": 0}]

# WRONG — skips items when removing
# for p in team:
#     if p['hp'] == 0:
#         team.remove(p)

# CORRECT — iterate over a copy
conscious = [p for p in team if p['hp'] > 0]
print(f"Conscious: {[p['name'] for p in conscious]}")

---

## **Practice Exercises**

### **Task 1: Read the Traceback**

Run the code, read the traceback, and explain in a comment what the error is and where it occurs.

**Expected Output:**
```
Pikachu's type: Electric
```

In [ ]:
pikachu = {"name": "Pikachu", "type": "Electric"}

# This has a bug — run it, read the error, then fix it:
# print(f"Pikachu's type: {pikachu['typ']}")  # Bug on purpose

# What is the error type?
# What does the message say?
# Fix:


### **Task 2: Add Print Debugging**

Add print statements inside the loop to find why the total is wrong.

**Expected Output (after fix):**
```
Total HP: 192
```

In [ ]:
team = [
    {"name": "Pikachu",  "hp": 35},
    {"name": "Charizard","hp": 78},
    {"name": "Blastoise","hp": 79},
]

def total_hp(team):
    total = 0
    for p in team:
        total = p['hp']   # Bug: should be +=
    return total

# Add debug prints, find the bug, fix it:


### **Task 3: diagnose_variable**

Use the `diagnose_variable` approach to inspect a mystery value before using it.

**Expected Output:**
```
Type: str — need to convert with int()
Level: 25
```

In [ ]:
mystery = "25"  # Came from user input

# Your diagnosis and fix here:


### **Task 4: Fix Off-by-One**

The loop prints one extra (or one fewer) Pokemon. Fix the range.

**Expected Output:**
```
0: Pikachu
1: Charizard
2: Blastoise
```

In [ ]:
team = ["Pikachu", "Charizard", "Blastoise"]

# Buggy loop — fix the range:
for i in range(1, len(team) + 1):
    print(f"{i}: {team[i]}")

### **Task 5: Fix Mutable Default**

The function shares state between calls. Fix the default argument.

**Expected Output:**
```
['Pikachu']
['Charizard']
```

In [ ]:
def add_to_team(pokemon, team=[]):
    team.append(pokemon)
    return team

# These should return separate lists — fix the function:
print(add_to_team("Pikachu"))
print(add_to_team("Charizard"))

### **Task 6: Minimal Test Case**

Extract the smallest snippet from a larger function that reproduces the bug.

**Expected Output:**
```
Bug: 8800
Fixed: 17.2
```

In [ ]:
# Isolate just the damage formula from a big function and test it:
attack = 55
defense = 160

# Bug: damage = attack * defense
# Fix: damage = attack / defense * 50

# Your code here:


### **Task 7: Rubber Duck Debugging**

Write comments explaining what each line of the buggy function does, then spot the error.

**Expected Output:**
```
Strongest: Charizard (Level 36)
```

In [ ]:
def find_strongest(team):
    strongest = None
    for p in team:
        if strongest is None or p['level'] > strongest:  # Bug!
            strongest = p
    return strongest

team = [
    {"name": "Pikachu",   "level": 25},
    {"name": "Charizard", "level": 36},
    {"name": "Rattata",   "level": 8},
]

# Add comments explaining each line, find the bug, fix it:


### **Task 8: Mutation Bug**

Fix the loop that skips Pokemon when removing fainted ones.

**Expected Output:**
```
Remaining: ['Charizard']
```

In [ ]:
team = [
    {"name": "Pikachu",  "hp": 0},
    {"name": "Charizard","hp": 45},
    {"name": "Blastoise","hp": 0},
]

# Buggy — modifying list while iterating:
# for p in team:
#     if p['hp'] == 0:
#         team.remove(p)

# Your fix here:


### **Task 9: Add Logging**

Replace the `print()` statements with `logging.debug()` and `logging.info()` calls.

**Expected Output:**
```
DEBUG    Healing Pikachu by 20
INFO     Pikachu HP: 35/35
```

In [ ]:
import logging
logging.basicConfig(level=logging.DEBUG, format='%(levelname)-8s %(message)s')

def heal(pokemon, amount):
    print(f"Healing {pokemon['name']} by {amount}")   # Replace with logging.debug
    pokemon['hp'] = min(pokemon['max_hp'], pokemon['hp'] + amount)
    print(f"{pokemon['name']} HP: {pokemon['hp']}/{pokemon['max_hp']}")  # Replace with logging.info

pikachu = {"name": "Pikachu", "hp": 15, "max_hp": 35}
heal(pikachu, 20)

### **Task 10: Full Debug Session**

The function below has at least two bugs. Use print debugging to find and fix them both.

**Expected Output:**
```
Winner: Charizard
```

In [ ]:
def find_winner(team1, team2):
    """Return the name of the team with the higher total level."""
    total1 = 0
    for p in team1:
        total1 = p['level']       # Bug 1

    total2 = 0
    for p in team2:
        total2 = p['level']       # Bug 1 (same)

    if total1 > total2:
        return team1[0]['name']   # Bug 2: should return "Team 1" or similar
    else:
        return team2[1]['name']   # Bug 2: team2[1] may not exist

team_ash = [{"name": "Pikachu", "level": 25}]
team_gary = [{"name": "Charizard", "level": 36}]

# Add debug prints, identify both bugs, fix them:


---

## **Summary**

- Follow the process: Reproduce → Isolate → Understand → Fix → Verify
- **print() debugging**: fast, universal — inspect variables at key points
- **type() and dir()**: check what type a variable actually is
- **Tracebacks**: read bottom-up — last line is the error type and message
- **Rubber duck**: explain code out loud to find logic errors
- **Minimal test case**: strip down to the smallest reproducing snippet
- **pdb / breakpoint()**: interactive debugger for complex bugs
- **logging**: professional print-debugging with levels and timestamps
- Common patterns: off-by-one, mutable defaults, mutation while iterating

---

## **Quick Reference**

```python
# Print debugging
print(f"DEBUG: var={var!r} type={type(var).__name__}")

# Inspect unknown variable
print(type(x), dir(x))

# Read traceback bottom-up:
# Last line  = error type + message  ← read this first
# Lines above = call chain           ← find your code

# pdb (interactive debugger)
breakpoint()   # Pause here
# n=next  s=step  c=continue  q=quit  p var=print var

# Logging
import logging
logging.basicConfig(level=logging.DEBUG)
logging.debug("detail")   # Dev-only detail
logging.info("event")     # Normal events
logging.warning("caution")# Something unexpected
logging.error("problem")  # Error handled

# Common bug patterns
range(len(x))       # not range(1, len(x)+1)
def f(lst=None):    # not def f(lst=[])
    if lst is None: lst = []
[x for x in lst if keep(x)]  # not: remove while iterating
```